In [ ]:
using CSV, DataFrames

df = CSV.read("aggregated_data_real.csv", DataFrame)

df = df[:, [:Census_Tract, :Num_lived, :Num_worked, :centroid_lat, :centroid_lon, :key_location]]

filtered_df = filter(row -> !ismissing(row.key_location), df)

size(df)[1]

324

In [2]:
df

Row,Census_Tract,Num_lived,Num_worked,centroid_lat,centroid_lon,key_location
,Int64,Int64,Int64,Float64,Float64,String?
1,5172,953,44344,42.3332,-83.0403,Ford Field
2,5173,716,2824,42.3457,-83.0505,missing
3,5324,489,18,42.3785,-83.0841,missing
4,5207,1116,10844,42.3347,-83.0555,missing
5,5170,952,275,42.3409,-83.0361,missing
6,5208,507,15685,42.3254,-83.0543,missing
7,5321,697,103,42.3863,-83.0916,Detroit Historical Museum
8,5171,517,267,42.3358,-83.033,missing
9,5043,520,24,42.4056,-82.9803,missing


In [3]:
home_loc= df[:,[:centroid_lat, :centroid_lon]] |> Tables.matrix;
work_loc= df[:,[:centroid_lat, :centroid_lon]] |> Tables.matrix;
station_potential_loc= df[:,[:centroid_lat, :centroid_lon]] |> Tables.matrix;
key_location_df= filtered_df[:,[:centroid_lat, :centroid_lon]] |> Tables.matrix;

#centers = CSV.File("HW3 data/centers.csv",header=0) |> Tables.matrix;

In [4]:
# Define indices
I = size(home_loc)[1] # Set of population density indices
J = size(work_loc)[1] # Set of job density indices
K = size(station_potential_loc)[1] # Set of candidate station indices
M = size(filtered_df)[1] # Set of key location indices

6

In [8]:
# Function to calculate Euclidean distance
function euclidean_distance(coord1, coord2)
    sqrt((coord1[1] - coord2[1])^2 + (coord1[2] - coord2[2])^2)
end

# Compute d1: Distance between population center i and station k
d1 = [euclidean_distance(home_loc[i, :], station_potential_loc[k, :]) for i in 1:I, k in 1:K]

# Compute d2: Distance between job center j and station k
d2 = [euclidean_distance(work_loc[j, :], station_potential_loc[k, :]) for j in 1:J, k in 1:K]

# Compute d3: Distance between key location m and station k
d3 = [euclidean_distance(key_location_df[m, :], station_potential_loc[k, :]) for m in 1:M, k in 1:K]

# Extract population at index i
p = df.Num_lived

# Extract number of workers at index j
w = df.Num_worked

station_number_limit = 25 # Maximum number of stations that can be built
key_location_distance_limit = 0.02 # Maximum distance for key locations to stations
M_large = 10000 # Large number for constraints

10000

In [9]:
using Gurobi
using JuMP

# Initialize the model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[k=1:K], Bin) # Binary variable: 1 if station k is built, 0 otherwise
@variable(model, a[i=1:I, k=1:K] >= 0) # People going from population center i to station k
@variable(model, b[j=1:J, k=1:K] >= 0) # People going from job center j to station k
@variable(model, c[m=1:M, k=1:K], Bin) # Binary variable: 1 if people go from key location m to station k

# Objective function
@objective(model, Min,
    sum(d1[i, k] * a[i, k] for i in 1:I, k in 1:K) +
    sum(d2[j, k] * b[j, k] for j in 1:J, k in 1:K)
)

# Constraints
@constraint(model, [i=1:I], sum(a[i, k] for k in 1:K) == p[i]) # Satisfy population demands
@constraint(model, [j=1:J], sum(b[j, k] for k in 1:K) == w[j]) # Satisfy job center demands
@constraint(model, [m=1:M], sum(c[m, k] for k in 1:K) == 1)     # Assign each key location to a station

@constraint(model, [i=1:I, k=1:K], a[i, k] <= M_large * x[k])  # Link a[i, k] with x[k]
@constraint(model, [j=1:J, k=1:K], b[j, k] <= M_large * x[k])  # Link b[j, k] with x[k]
@constraint(model, [m=1:M, k=1:K], c[m, k] <= M_large * x[k])  # Link c[m, k] with x[k]

@constraint(model, sum(x[k] for k in 1:K) <= station_number_limit) # Limit number of stations

@constraint(model, [m=1:M, k=1:K], d3[m, k] * c[m, k] <= key_location_distance_limit) # Limit distance for key locations

# Solve the model
optimize!(model)

# Output results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found!")
    println("Objective value: ", objective_value(model))
    println("Station decisions: ", value.(x))
else
    println("No optimal solution found.")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 214495 rows, 212220 columns and 637950 nonzeros
Model fingerprint: 0x8e1da80b
Variable types: 209952 continuous, 2268 integer (2268 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+04]
  Objective range  [4e-03, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 4e+04]
Presolve removed 3846 rows and 1902 columns
Presolve time: 2.21s
Presolved: 210649 rows, 210318 columns, 630306 nonzeros
Variable types: 209952 continuous, 366 integer (366 binary)
Found heuristic solution: objective 58134.674423
Found heuristic solution: objective 31446.982551
Deterministic concurrent LP optimizer: primal simplex, dual simpl

In [10]:
# Get the results
println("Objective value (Total distance walked from home to station and station to office): ", objective_value(model))

thestations=[]

# Selected landfills
for k in 1:K
    if value(x[k]) > 0.5
        println("station ", k, " is built.")
        push!(thestations, k)
    end
end

Objective value (Total distance walked from home to station and station to office): 11480.814539945402
station 1 is built.
station 4 is built.
station 6 is built.
station 8 is built.
station 31 is built.
station 56 is built.
station 60 is built.
station 88 is built.
station 115 is built.
station 153 is built.
station 167 is built.
station 188 is built.
station 212 is built.
station 224 is built.
station 235 is built.
station 250 is built.
station 263 is built.
station 272 is built.
station 295 is built.
station 296 is built.
station 300 is built.
station 312 is built.
station 315 is built.
station 321 is built.
station 322 is built.


In [28]:
#11480 - 25 stations
#6727 - 50 stations 

In [11]:
df.closest .= 0
df.closesteucdist .= 9999999999.0
for obs in 1:size(df,1)
    for station in 1:length(thestations)
        if euclidean_distance(df[obs,[:centroid_lat, :centroid_lon]],df[thestations[station],[:centroid_lat, :centroid_lon]]) < df.closesteucdist[obs]
            df.closest[obs]=thestations[station]
            df.closesteucdist[obs]=euclidean_distance(df[obs,[:centroid_lat, :centroid_lon]],df[thestations[station],[:centroid_lat, :centroid_lon]])
        end
    end
end

In [13]:
df.Num_lived_all .= 0
df.Num_worked_all .= 0

for obs in 1:size(df,1)
    df.Num_lived_all[df.closest[obs]]=df.Num_lived_all[df.closest[obs]]+df.Num_lived[obs]
    df.Num_worked_all[df.closest[obs]]=df.Num_worked_all[df.closest[obs]]+df.Num_worked[obs]
end